In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('/home/ajit.mehta/Ajit_work/phenom_hh/src/')
sys.path.append('../src/')
import template_22 as phhsi
from pycbc  import  detector

/home/sumitk/src/pycbc/local/lib/python2.7/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
def lnlike(param_vec, data, freq, psd, f_low, f_cut):
        """
        compute the log likelihood
        
        inputs: 
        param_vec : vector of parameters 
        dr, di, 
        freq : Fourier freq 
        psd : psd vector 
        flow,fcut
        
        output: 
        log_likelhood 
        """
        df = np.mean(np.diff(freq))

        N_low=np.int((f_low-freq[0])/df)
        N_cut=np.int((f_cut-freq[0])/df)

        Nls=np.int(f_low/df)  #N_low_signal
        Ncs=np.int(f_cut/df)  #N_cut_signal

        # unpacking the parameter vector 
        Mc, q, dL, i, t0, phi0, ra, sin_dec, pol = param_vec

        # generate the waveform 
        f, hpf, hcf = phhsi.phenomhh_waveform_SI(Mc, q, dL, i, t0, (phi0 %(2.*np.pi)), f_low, df, Ncs)

        # compute antenna patterns 
        Fp,Fc = detector.overhead_antenna_pattern(ra, np.arcsin(sin_dec), pol)

        signal=Fp*hpf+Fc*hcf

        like = -2.*df*np.real(np.dot(data[N_low:N_cut]-signal[Nls:Ncs],np.conj((data[N_low:N_cut]-signal[Nls:Ncs])/psd[N_low:N_cut])))

        return like#log-likelihood
    
def lnprior(param_vec):
        Mc, q, dL, i, t0, phi_0, ra, sin_dec, pol = param_vec
        if 1 < Mc < 200 and 0.05 < q <= 1. and  1.<dL<10000 and 0.<= i <= pi and -15.<= t0 <= 15. and -pi <= phi_0 <= 3.*pi and 0. <= ra < 2.*pi and -1. <= sin_dec <= 1. and 0. <= pol <= pi:
                return 2.*np.log(dL)+np.log(np.sin(i))
        return -np.inf

def lnprob(param_vec):
        lp = lnprior(param_vec)
        if not np.isfinite(lp):
                return -np.inf
        return lp + lnlike(param_vec, data, freq, psd, f_low, f_cut)

In [4]:
#data
freq, dr, di, psd = np.loadtxt('../injections/nsbh_sxs_20181209/BBH_M_140.00_iota_0.000_pol_0.000_t0_0_data.dat', unpack=True)
data = dr + 1j*di

#best-fit phenom_hm waveform

#parameters corresponding to best-fit phenom_hm waveform
mc_inj, q_inj, dL_inj, iota_inj, t0_inj, phi0_inj, ra_inj, sin_dec_inj, pol_inj = np.loadtxt('../injections/nsbh_sxs_20181209/BBH_M_140.00_iota_0.000_pol_0.000_t0_0_initial.dat', unpack=True)

f_low = 20.
f_cut = 999.
df = np.mean(np.diff(freq))
Ncs=np.int(f_cut/df)
f, hpf, hcf = phhsi.phenomhh_waveform_SI(mc_inj, q_inj, dL_inj, iota_inj, t0_inj, (phi0_inj %(2.*np.pi)), f_low, df, Ncs)
Fp,Fc = detector.overhead_antenna_pattern(ra_inj, np.arcsin(sin_dec_inj), pol_inj)
best_fit_signal=Fp*hpf+Fc*hcf

plt.figure(figsize=(10,5))
plt.subplot(121)
plt.loglog(freq, abs(data), label='SXS')
plt.loglog(f, abs(best_fit_signal), label='phenomhm')
plt.loglog(freq, np.sqrt(psd), label='PSD')
plt.xlim([f_low, 128.])
plt.ylim([1e-24, 1e-22])
plt.legend(loc='best')
plt.subplot(122)
plt.title('zoomed-out')
plt.loglog(freq, abs(data))
plt.loglog(f, abs(best_fit_signal))
plt.loglog(freq, np.sqrt(psd))
plt.ylim([1e-30, 1e-21])
plt.show()

plt.figure(figsize=(10,5))
plt.subplot(121)
plt.loglog(freq, np.real(data), label='SXS', alpha=0.5)
plt.loglog(f, np.imag(best_fit_signal), label='phenomhm', alpha=0.5)
plt.ylabel('real(hf)')
plt.ylim([1e-27, 1e-22])
plt.xlim([f_low, 128])
plt.legend(loc='best')
plt.subplot(122)
plt.loglog(freq, np.imag(data), label='SXS', alpha=0.5)
plt.loglog(f, np.real(best_fit_signal), label='phenomhm', alpha=0.5)
plt.ylim([1e-27, 1e-22])
plt.xlim([f_low, 128])
plt.ylabel('imag(hf)')
plt.show()

IOError: [Errno 2] No such file or directory: '../injections/nsbh_sxs_20181209/BBH_M_140.00_iota_0.000_pol_0.000_t0_0_data.dat'

In [5]:
param_set = ['mc','q']
lnlike_data_map = {'mc':[],'q':[]}
lnlike_bfs_map = {'mc':[],'q':[]}
param_range_map = {'mc':np.linspace(35,45,50),'q':np.linspace(0.1,0.2,50)}

plt.figure(figsize=(10,5))
for (idx, param) in enumerate(param_set):
    param_range = param_range_map[param]
    for x in param_range:
        param_vec = mc_inj, q_inj, dL_inj, iota_inj, t0_inj, phi0_inj, ra_inj, sin_dec_inj, pol_inj
        param_vec = list(param_vec)
        param_vec[idx] = x
        param_vec = tuple(param_vec)
        lnlike_data_map[param].append(lnlike(param_vec, data, freq, psd, f_low, f_cut))
        lnlike_bfs_map[param].append(lnlike(param_vec, best_fit_signal, f, psd, f_low, f_cut))
    plt.subplot(1,2,idx+1)
    plt.plot(param_range_map[param], lnlike_data_map[param])
    plt.plot(param_range_map[param], lnlike_bfs_map[param])
    plt.xlabel(param)
    plt.ylabel('lnlike')
    
plt.show()    

NameError: name 'mc_inj' is not defined